In [0]:
# import libraries
import pyspark.sql.functions as F
from pyspark.sql.types import *
from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


from pyspark.sql import functions as f
from pyspark.sql import SQLContext
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql.functions import isnan, when, count, col, isnull, percent_rank, avg, mean
from pyspark.sql.functions import min
from pyspark.sql.functions import col, max
from pyspark.sql.functions import format_string
from pyspark.sql.functions import substring
from pyspark.sql.functions import concat_ws
from pyspark.sql.functions import concat
from pyspark.sql.functions import to_timestamp
from pyspark.sql.functions import lit
from pyspark.sql.functions import to_utc_timestamp
from pyspark.sql.functions import expr
from pyspark.sql.functions import regexp_replace
from pyspark.sql.functions import instr
from pyspark.sql.functions import row_number
from pyspark.sql.window import Window
from pyspark.sql.types import IntegerType

from pyspark.ml.linalg import DenseVector, SparseVector, Vectors
from pyspark.ml.feature import VectorAssembler, StandardScaler, StringIndexer,OneHotEncoder
from pyspark.ml.classification import MultilayerPerceptronClassifier

from pyspark.ml import Pipeline
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.classification import GBTClassifier

from pyspark.ml.classification import RandomForestClassifier, DecisionTreeClassifier, LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator

In [0]:
#Initializes blob storage credentials/location
blob_container = "w261-sec4-group2" # The name of your container created in https://portal.azure.com
storage_account = "kdevery" # The name of your Storage account created in https://portal.azure.com
secret_scope = "sec4-group2" # The name of the scope created in your local computer using the Databricks CLI
secret_key = "w261-key" # The name of the secret key created in your local computer using the Databricks CLI 
blob_url = f"wasbs://{blob_container}@{storage_account}.blob.core.windows.net"
mount_path = "/mnt/mids-w261"

#Points to SAS token
spark.conf.set(
  f"fs.azure.sas.{blob_container}.{storage_account}.blob.core.windows.net",
  dbutils.secrets.get(scope = secret_scope, key = secret_key)
)

In [0]:
display(dbutils.fs.ls(f"{blob_url}"))

path,name,size,modificationTime
wasbs://w261-sec4-group2@kdevery.blob.core.windows.net/MLPC_predictions_df/,MLPC_predictions_df/,0,1670111625000
wasbs://w261-sec4-group2@kdevery.blob.core.windows.net/feature_engineered_data/,feature_engineered_data/,0,1668924639000
wasbs://w261-sec4-group2@kdevery.blob.core.windows.net/feature_engineered_data_test/,feature_engineered_data_test/,0,1668924670000
wasbs://w261-sec4-group2@kdevery.blob.core.windows.net/feature_engineered_train_data/,feature_engineered_train_data/,0,1668559613000
wasbs://w261-sec4-group2@kdevery.blob.core.windows.net/merged_cleaned_data/,merged_cleaned_data/,0,1669494945000
wasbs://w261-sec4-group2@kdevery.blob.core.windows.net/merged_cleaned_data_test/,merged_cleaned_data_test/,0,1669495012000
wasbs://w261-sec4-group2@kdevery.blob.core.windows.net/merged_cleaned_data_train/,merged_cleaned_data_train/,0,1669495000000
wasbs://w261-sec4-group2@kdevery.blob.core.windows.net/merged_data/,merged_data/,0,1669494746000
wasbs://w261-sec4-group2@kdevery.blob.core.windows.net/number_flights_and_delay_rate/,number_flights_and_delay_rate/,0,1669961275000
wasbs://w261-sec4-group2@kdevery.blob.core.windows.net/pagerank_scores/,pagerank_scores/,0,1669963319000


In [0]:
#Read processed folds from the blob

d_processed = {}

d_processed['df1'] = spark.read.parquet(f"{blob_url}/processed_fold_1")
d_processed['df2'] = spark.read.parquet(f"{blob_url}/processed_fold_2")
d_processed['df3'] = spark.read.parquet(f"{blob_url}/processed_fold_3")
d_processed['df4'] = spark.read.parquet(f"{blob_url}/processed_fold_4")
d_processed['df5'] = spark.read.parquet(f"{blob_url}/processed_fold_5")

In [0]:
display(d_processed['df1'])

label,scaled_feature_vector,cv
0.0,"Map(vectorType -> sparse, length -> 38, indices -> List(0, 1, 2, 3, 14, 29, 30, 34, 35, 36, 37), values -> List(0.2829102732363673, 0.11328696931622412, 2.0113782845276376, 2.6100149853683066, 3.50688413374736, 1.9267153947847935, 0.024454135622888753, 5.800083605037825, 6.548264560151223, 6.868844597961218, 5.689466238716571))",train
0.0,"Map(vectorType -> sparse, length -> 38, indices -> List(0, 1, 2, 3, 19, 29, 30, 34, 35, 36, 37), values -> List(0.2829102732363673, 0.11328696931622412, 2.0113782845276376, 2.1966940003036726, 6.993165165142769, 1.9267153947847935, 0.024454135622888753, 7.179892852856789, 6.6576549643405665, 5.354399633706597, 4.708838137504222))",train
0.0,"Map(vectorType -> sparse, length -> 38, indices -> List(0, 1, 2, 3, 11, 29, 30, 34, 35, 36, 37), values -> List(0.2829102732363673, 0.11328696931622412, 2.0113782845276376, 2.6182484711265666, 2.765871182690818, 1.9267153947847935, 0.2676505143925165, 3.207577237836827, 3.8393605659437378, 4.622677367487146, 4.243782105408468))",train
0.0,"Map(vectorType -> sparse, length -> 38, indices -> List(0, 1, 2, 3, 19, 30, 34, 35, 36, 37), values -> List(0.2829102732363673, 0.11328696931622412, 2.0113782845276376, 2.4486386645064178, 6.993165165142769, 4.406792871000435, 4.00384742041291, 4.1130235004661415, 5.354399633706597, 4.708838137504222))",train
0.0,"Map(vectorType -> sparse, length -> 38, indices -> List(0, 1, 2, 3, 11, 29, 30, 34, 35, 36, 37), values -> List(0.2829102732363673, 0.11328696931622412, 2.0113782845276376, 1.9743898848306622, 2.765871182690818, 1.9267153947847935, 1.273968411912448, 6.979046048945935, 6.557782615673814, 4.622677367487146, 4.243782105408468))",train
0.0,"Map(vectorType -> sparse, length -> 38, indices -> List(0, 1, 2, 3, 19, 30, 34, 35, 36, 37), values -> List(0.2829102732363673, 0.11328696931622412, 2.0113782845276376, 1.936515850342668, 6.993165165142769, 0.045240150902344185, 4.838324829454176, 3.161186587884443, 5.354399633706597, 4.708838137504222))",train
0.0,"Map(vectorType -> sparse, length -> 38, indices -> List(0, 1, 2, 3, 16, 29, 30, 34, 35, 36, 37), values -> List(0.2829102732363673, 0.11328696931622412, 2.0113782845276376, 1.8591210842150274, 4.930679233265689, 1.9267153947847935, 0.024454135622888753, 5.800083605037825, 6.548264560151223, 5.5351753891683675, 4.465800083054335))",train
1.0,"Map(vectorType -> sparse, length -> 38, indices -> List(0, 1, 2, 3, 19, 30, 34, 35, 36, 37), values -> List(0.2829102732363673, 0.11328696931622412, 2.0113782845276376, 2.371243898378777, 6.993165165142769, 0.20841037084606018, 5.174030424982984, 4.730480463338929, 7.388809045080336, 6.2894652840063685))",train
0.0,"Map(vectorType -> sparse, length -> 38, indices -> List(0, 1, 2, 3, 17, 29, 30, 34, 35, 36, 37), values -> List(0.2829102732363673, 0.11328696931622412, 2.0113782845276376, 2.3844174755919925, 5.804648793558455, 1.9267153947847935, 0.19150354566816324, 4.116696920148476, 4.441151632931046, 3.26679730247171, 3.0817728258802504))",train
0.0,"Map(vectorType -> sparse, length -> 38, indices -> List(0, 1, 2, 3, 12, 29, 30, 34, 35, 36, 37), values -> List(0.2829102732363673, 0.11328696931622412, 2.0113782845276376, 3.8368043633489934, 2.9494046905439393, 1.9267153947847935, 0.3947907751011952, 5.175341559775867, 4.977681941785401, 5.854295325287253, 5.0531772415694105))",train


#Cross Validation and GridSearch for Logistic Regression

In [0]:
%run "/Shared/w261_Section4_Group2/Phase 3/custom_cv_module"

#### Logistic Regression

In [0]:
# set up grid search: estimator, set of params, and evaluator
logistic_model = LogisticRegression(labelCol="label", featuresCol="scaled_feature_vector")
grid = ParamGridBuilder()\
            .addGrid(logistic_model.threshold, [0.3,0.5,0.8])\
            .addGrid(logistic_model.regParam, [0.01,0.1,0.5,1.0, 2.0])\
            .addGrid(logistic_model.elasticNetParam, [0.0,0.25,0.50,0.75, 1.0])\
            .addGrid(logistic_model.maxIter, [1,5,10,20, 50])\
            .build() 

# Example using F0.5 score for evaluator
evaluator = MulticlassClassificationEvaluator(metricName='fMeasureByLabel', beta=0.5, metricLabel=1)

In [0]:
# run cross validation & return the crossvalidation F0.5 score for 'test' set
cv = CustomCrossValidator(estimator=logistic_model, estimatorParamMaps=grid, evaluator=evaluator,splitWord =('train','val'), cvCol = 'cv',parallelism=10)

In [0]:
cvModel = cv.fit(d_processed)

fold 1 start...
fold 1 end
fold 2 start...
fold 2 end
fold 3 start...
fold 3 end
fold 4 start...
fold 4 end
fold 5 start...
fold 5 end
Best Model:  {Param(parent='LogisticRegression_fa2c191371e5', name='threshold', doc='Threshold in binary classification prediction, in range [0, 1]. If threshold and thresholds are both set, they must match.e.g. if threshold is p, then thresholds must be equal to [1-p, p].'): 0.3, Param(parent='LogisticRegression_fa2c191371e5', name='regParam', doc='regularization parameter (>= 0).'): 0.01, Param(parent='LogisticRegression_fa2c191371e5', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 1.0, Param(parent='LogisticRegression_fa2c191371e5', name='maxIter', doc='max number of iterations (>= 0).'): 5} Detailed Score [0.6137074715214731, 0.6142862797265135, 0.612445455600374, 0.602203958042278, 0.5590722107293431] Avg Score 0.6003430751239963


In [0]:
#for individual testing

#test_train = d_processed['df1'].filter(col('cv')=='train')
#test_val = d_processed['df1'].filter(col('cv')=='val')

#test_logistic_model = LogisticRegression(labelCol="label", featuresCol="scaled_feature_vector")
#evaluator = MulticlassClassificationEvaluator(metricName='accuracy')
#lrModel = test_logistic_model.fit(test_train)
#predictions = lrModel.transform(test_val)
#evaluator.evaluate(predictions)

In [0]:
# make predictions
predictions = cvModel.transform(d_processed['df1'])

display(predictions.groupby('label', 'prediction').count())

label,prediction,count
1.0,1.0,527655
0.0,1.0,235200
1.0,0.0,743313
0.0,0.0,5567162


In [0]:
fbeta = cvModel.avgMetrics[1]
print(f"Logistic Regression F0.5 Score: {fbeta}")

Logistic Regression F0.5 Score: 0.5911200015965978


#Full Train and Test Sets

In [0]:
# Read in processed training and test data

processed_train_df = spark.read.parquet(f"{blob_url}/processed_train")
processed_test_df = spark.read.parquet(f"{blob_url}/processed_test")

In [0]:
processed_train_df.count()

Out[8]: 35366654

In [0]:
processed_test_df.count()

Out[9]: 5859306

In [0]:
display(processed_test_df)

label,scaled_feature_vector,index
0.0,"Map(vectorType -> sparse, length -> 39, indices -> List(0, 1, 2, 3, 4, 9, 22, 30, 31, 33, 35, 36, 37, 38), values -> List(0.6062867010278674, 0.4552178936348829, 1.9962981366261658, 1.1771165614071244, 1.1807240188784864, 2.001448286999518, 4.804541102597839, 0.5290014255667469, 2.5358333967474525, 0.3529775526800816, 2.2518971367325515, 0.2623600525700863, 2.453487744696793, 0.4321511055947388))",0
0.0,"Map(vectorType -> sparse, length -> 39, indices -> List(0, 1, 2, 3, 4, 9, 21, 30, 31, 33, 35, 36, 37, 38), values -> List(0.6062867010278674, 0.4552178936348829, 1.9962981366261658, 0.8181980723075531, 1.518073738558054, 2.001448286999518, 5.254269120180973, 0.34977829198919363, 0.9689112552858259, 0.433041426600483, 2.3371183222456193, 0.27964920300085777, 2.453487744696793, 0.4321511055947388))",1
0.0,"Map(vectorType -> sparse, length -> 39, indices -> List(0, 1, 2, 3, 4, 9, 21, 30, 31, 33, 35, 36, 37, 38), values -> List(0.6062867010278674, 0.4552178936348829, 1.9962981366261658, 0.8181980723075531, 1.1807240188784864, 2.001448286999518, 5.254269120180973, 0.4050167164495406, 0.9689112552858259, 0.433041426600483, 2.3371183222456193, 0.27964920300085777, 2.453487744696793, 0.4321511055947388))",2
0.0,"Map(vectorType -> sparse, length -> 39, indices -> List(0, 1, 2, 3, 9, 21, 30, 31, 33, 35, 36, 37, 38), values -> List(0.6062867010278674, 0.4552178936348829, 1.9962981366261658, 0.8181980723075531, 2.001448286999518, 5.254269120180973, 0.34688490538266537, 0.9689112552858259, 0.433041426600483, 2.3371183222456193, 0.27964920300085777, 2.453487744696793, 0.4321511055947388))",3
0.0,"Map(vectorType -> sparse, length -> 39, indices -> List(0, 1, 2, 3, 4, 9, 22, 30, 31, 33, 35, 36, 37, 38), values -> List(0.6062867010278674, 0.4552178936348829, 1.9962981366261658, 0.722940084679231, 2.361448037756973, 2.001448286999518, 4.804541102597839, 0.38154845811321203, 0.7047198828462189, 0.5403663049134857, 1.8916360906620093, 0.4360534659923761, 2.453487744696793, 0.4321511055947388))",4
0.0,"Map(vectorType -> sparse, length -> 39, indices -> List(0, 1, 2, 3, 4, 9, 22, 30, 31, 33, 35, 36, 37, 38), values -> List(0.6062867010278674, 0.4552178936348829, 1.9962981366261658, 0.722940084679231, 0.8433742991989189, 2.001448286999518, 4.804541102597839, 0.3154302374467945, 0.7047198828462189, 0.5403663049134857, 1.8916360906620093, 0.4360534659923761, 2.453487744696793, 0.4321511055947388))",5
1.0,"Map(vectorType -> sparse, length -> 39, indices -> List(0, 1, 2, 3, 4, 7, 9, 22, 30, 31, 33, 35, 36, 37, 38), values -> List(0.6062867010278674, 0.4552178936348829, 1.9962981366261658, 0.722940084679231, 2.1927731779171893, 10.563083864358715, 2.001448286999518, 4.804541102597839, 0.608351639401759, 0.7047198828462189, 0.5403663049134857, 1.8916360906620093, 0.4360534659923761, 2.453487744696793, 0.4321511055947388))",6
0.0,"Map(vectorType -> sparse, length -> 39, indices -> List(0, 1, 2, 3, 4, 26, 30, 31, 35, 36, 37, 38), values -> List(0.6062867010278674, 0.4552178936348829, 1.9962981366261658, 1.5003133051460746, 0.5060245795193513, 7.296613042424572, 1.3850953356442304, 1.5473254271491774, 2.506155999639365, 0.1854260324295884, 2.453487744696793, 0.4321511055947388))",7
0.0,"Map(vectorType -> sparse, length -> 39, indices -> List(0, 1, 2, 3, 4, 26, 30, 31, 35, 36, 37, 38), values -> List(0.6062867010278674, 0.4552178936348829, 1.9962981366261658, 1.6500044285620095, 1.0120491590387026, 7.296613042424572, 1.5574891757029543, 1.0216639197618846, 3.020166622012421, 0.35300129827896976, 2.453487744696793, 0.4321511055947388))",8
1.0,"Map(vectorType -> sparse, length -> 39, indices -> List(0, 1, 2, 3, 4, 9, 21, 30, 31, 33, 35, 36, 37, 38), values -> List(1.5157167525696686, 1.251849207495928, 0.9981490683130829, 0.8181980723075531, 1.0120491590387026, 2.001448286999518, 5.254269120180973, 1.5143994442576951, 0.9689112552858259, 0.756889388763837, 2.374154468913133, 0.4757177694132144, 2.364497822755491, 0.2879023

In [0]:
#regParam=0.001, elasticNetParam=1,

final_logistic_model = LogisticRegression(labelCol="label", featuresCol="scaled_feature_vector", threshold=0.3, regParam=0.01, elasticNetParam=1.0, maxIter=5)
evaluator = MulticlassClassificationEvaluator(metricName='fMeasureByLabel', beta=0.5, metricLabel=1)
lrModel = final_logistic_model.fit(processed_train_df)

In [0]:
predictions = lrModel.transform(processed_test_df)

In [0]:
display(predictions)

label,scaled_feature_vector,index,rawPrediction,probability,prediction
0.0,"Map(vectorType -> sparse, length -> 39, indices -> List(0, 1, 2, 3, 4, 9, 22, 30, 31, 33, 35, 36, 37, 38), values -> List(0.6062867010278674, 0.4552178936348829, 1.9962981366261658, 1.1771165614071244, 1.1807240188784864, 2.001448286999518, 4.804541102597839, 0.5290014255667469, 2.5358333967474525, 0.3529775526800816, 2.2518971367325515, 0.2623600525700863, 2.453487744696793, 0.4321511055947388))",0,"Map(vectorType -> dense, length -> 2, values -> List(2.6953389679742576, -2.6953389679742576))","Map(vectorType -> dense, length -> 2, values -> List(0.9367510462130532, 0.06324895378694684))",0.0
0.0,"Map(vectorType -> sparse, length -> 39, indices -> List(0, 1, 2, 3, 4, 9, 21, 30, 31, 33, 35, 36, 37, 38), values -> List(0.6062867010278674, 0.4552178936348829, 1.9962981366261658, 0.8181980723075531, 1.518073738558054, 2.001448286999518, 5.254269120180973, 0.34977829198919363, 0.9689112552858259, 0.433041426600483, 2.3371183222456193, 0.27964920300085777, 2.453487744696793, 0.4321511055947388))",1,"Map(vectorType -> dense, length -> 2, values -> List(2.726066858945747, -2.726066858945747))","Map(vectorType -> dense, length -> 2, values -> List(0.9385473794676553, 0.06145262053234468))",0.0
0.0,"Map(vectorType -> sparse, length -> 39, indices -> List(0, 1, 2, 3, 4, 9, 21, 30, 31, 33, 35, 36, 37, 38), values -> List(0.6062867010278674, 0.4552178936348829, 1.9962981366261658, 0.8181980723075531, 1.1807240188784864, 2.001448286999518, 5.254269120180973, 0.4050167164495406, 0.9689112552858259, 0.433041426600483, 2.3371183222456193, 0.27964920300085777, 2.453487744696793, 0.4321511055947388))",2,"Map(vectorType -> dense, length -> 2, values -> List(2.720960236536834, -2.720960236536834))","Map(vectorType -> dense, length -> 2, values -> List(0.9382521884753738, 0.06174781152462616))",0.0
0.0,"Map(vectorType -> sparse, length -> 39, indices -> List(0, 1, 2, 3, 9, 21, 30, 31, 33, 35, 36, 37, 38), values -> List(0.6062867010278674, 0.4552178936348829, 1.9962981366261658, 0.8181980723075531, 2.001448286999518, 5.254269120180973, 0.34688490538266537, 0.9689112552858259, 0.433041426600483, 2.3371183222456193, 0.27964920300085777, 2.453487744696793, 0.4321511055947388))",3,"Map(vectorType -> dense, length -> 2, values -> List(2.774251567139633, -2.774251567139633))","Map(vectorType -> dense, length -> 2, values -> List(0.941268463697446, 0.05873153630255401))",0.0
0.0,"Map(vectorType -> sparse, length -> 39, indices -> List(0, 1, 2, 3, 4, 9, 22, 30, 31, 33, 35, 36, 37, 38), values -> List(0.6062867010278674, 0.4552178936348829, 1.9962981366261658, 0.722940084679231, 2.361448037756973, 2.001448286999518, 4.804541102597839, 0.38154845811321203, 0.7047198828462189, 0.5403663049134857, 1.8916360906620093, 0.4360534659923761, 2.453487744696793, 0.4321511055947388))",4,"Map(vectorType -> dense, length -> 2, values -> List(2.736400988761506, -2.736400988761506))","Map(vectorType -> dense, length -> 2, values -> List(0.9391407184679412, 0.06085928153205877))",0.0
0.0,"Map(vectorType -> sparse, length -> 39, indices -> List(0, 1, 2, 3, 4, 9, 22, 30, 31, 33, 35, 36, 37, 38), values -> List(0.6062867010278674, 0.4552178936348829, 1.9962981366261658, 0.722940084679231, 0.8433742991989189, 2.001448286999518, 4.804541102597839, 0.3154302374467945, 0.7047198828462189, 0.5403663049134857, 1.8916360906620093, 0.4360534659923761, 2.453487744696793, 0.4321511055947388))",5,"Map(vectorType -> dense, length -> 2, values -> List(2.802478211117589, -2.802478211117589))","Map(vectorType -> dense, length -> 2, values -> List(0.942809595136699, 0.05719040486330096))",0.0
1.0,"Map(vectorType -> sparse, length -> 39, indices -> List(0, 1, 2, 3, 4, 7, 9, 22, 30, 31, 33, 35, 36, 37, 38), values -> List(0.6062867010278674, 0.4552178936348829, 1.9962981366261658, 0.722940084679231, 2.1927731779171893, 10.563083864358715, 2.001448286999518, 4.804541102597839, 0.608351639401759, 0.7047198828462189, 0.5

In [0]:
display(predictions.groupby('label', 'prediction').count())

label,prediction,count
1.0,1.0,385411
0.0,1.0,175635
1.0,0.0,633292
0.0,0.0,4664968


In [0]:
evaluator.evaluate(predictions)

Out[15]: 0.5905981420747944

In [0]:
# feature_cols = ['MONTH','DAY_OF_MONTH','DAY_OF_WEEK','DISTANCE','HourlyWindSpeed','Rain','Blowing','Snow','Thunder','CloudySkyCondition','carrier_vec',         'holiday_period','mean_carrier_delay','Pagerank_Score','PREV_FLIGHT_DELAYED','origin_percent_delayed','dest_percent_delayed','ORIGIN_Prophet_trend','ORIGIN_Prophet_pred','DEST_Prophet_trend','DEST_Prophet_pred']

lrModel.coefficients


Out[16]: SparseVector(39, {3: 0.0014, 4: 0.0312, 5: 0.0296, 7: 0.0109, 8: 0.0723, 9: 0.007, 30: 0.283, 32: 0.7632, 33: 0.0127, 34: 0.0055, 35: 0.1052, 37: 0.0149})

In [0]:
predictions.write.mode("overwrite").parquet(f"{blob_url}/predictions_LR")